<a href="https://colab.research.google.com/github/EshwarDhande/Model-Tweaks/blob/main/Quickplay.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:

!pip install transformers

from transformers import AutoModel, AutoTokenizer

model_name = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


/home/eshwar.dhande/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/eshwar.dhande/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/eshwar.dhande/miniconda3/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
print(model)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [4]:
seventh_layer = model.encoder.layer[6]

In [5]:
attention_weights = seventh_layer.attention.self.query.weight
attention_biases = seventh_layer.attention.self.query.bias

print("Original attention weights shape:", attention_weights.shape)
print("Original attention biases shape:", attention_biases.shape)


Original attention weights shape: torch.Size([768, 768])
Original attention biases shape: torch.Size([768])


In [6]:
print("Original attention weights :", attention_weights)
print("Original attention biases :", attention_biases)

Original attention weights : Parameter containing:
tensor([[-0.0253,  0.0489, -0.0148,  ...,  0.0431, -0.0253,  0.0013],
        [-0.0143, -0.0465, -0.0020,  ..., -0.0073, -0.0296,  0.0208],
        [ 0.0147, -0.0400, -0.0344,  ..., -0.0697, -0.1240, -0.0031],
        ...,
        [-0.0214, -0.0160, -0.0680,  ...,  0.0233, -0.0359,  0.0355],
        [-0.0297,  0.0270,  0.0464,  ...,  0.0691,  0.0293,  0.0095],
        [-0.0012, -0.0403,  0.0343,  ..., -0.0742,  0.0371,  0.0421]],
       requires_grad=True)
Original attention biases : Parameter containing:
tensor([-1.7320e-01,  2.7564e-01,  1.6801e-01, -1.0946e-01,  1.6433e-01,
        -1.1501e-01, -2.5965e-01, -1.3676e-01, -3.0756e-01, -4.5222e-01,
         1.0455e-01, -8.2313e-02, -9.7356e-02, -2.0522e-02,  1.1782e-01,
         7.8542e-02, -1.1049e-01,  8.0185e-02,  3.4537e-02, -6.8996e-02,
        -6.8539e-04,  2.3121e-01,  1.2190e-01,  1.4254e-01, -7.6825e-02,
         1.5386e-01,  9.6756e-04,  1.0246e-01,  1.4043e-01, -1.5966e-01,


In [7]:
random_weights = torch.randn_like(attention_weights)
random_biases = torch.randn_like(attention_biases)


In [8]:
seventh_layer.attention.self.query.weight.data = random_weights
seventh_layer.attention.self.query.bias.data = random_biases

In [9]:
print("Modified attention weights:", seventh_layer.attention.self.query.weight)
print("Modified attention biases:", seventh_layer.attention.self.query.bias)

Modified attention weights: Parameter containing:
tensor([[ 1.1698,  0.2641,  0.3018,  ...,  1.2650, -0.6742, -0.6350],
        [-0.4757,  1.1112,  0.3508,  ..., -1.0191,  1.0730,  0.0345],
        [ 0.7104,  1.6383, -0.2439,  ...,  0.7870,  1.2489,  0.2052],
        ...,
        [-1.1359, -1.4630, -0.1698,  ...,  1.8636, -0.2108, -0.8792],
        [ 0.9811,  0.7351,  1.4844,  ...,  0.2921,  0.9508, -0.7346],
        [-0.0873, -0.3575, -1.3820,  ...,  0.6005, -0.3451,  1.7388]],
       requires_grad=True)
Modified attention biases: Parameter containing:
tensor([ 1.0830e+00,  8.8136e-01,  1.9859e-02, -1.8647e+00,  1.6159e+00,
         1.2092e-01, -2.8872e-01,  3.2829e-01,  2.7526e-01, -6.8923e-01,
        -5.8437e-01,  2.0669e+00,  1.4716e+00,  2.6694e+00, -1.1395e+00,
         5.4449e-01,  8.8057e-01, -5.6363e-01, -2.6202e-01,  2.5452e-01,
         9.7497e-01,  1.1441e+00, -1.6501e+00,  1.8174e+00, -2.7642e-01,
        -5.1492e-01,  1.7267e+00, -3.7638e-01,  4.4722e-01, -2.4279e-01,
  